<a href="https://colab.research.google.com/github/Keoni808/NFL_Data_Cleaning/blob/main/NFL_Plays_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mounting and Imports

In [2]:
# Mount your Google Drive to access files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [11]:
# imports

# Data manipulation
import pandas as pd

# Regular expressions
import re

# Grab data from database
from google.cloud import bigquery

# Loading Data


In [5]:
# Client connect to bigquery project
client = bigquery.Client('nfl-data-430702')

In [6]:
nfl_2023_plays_query = """
                       SELECT *
                       FROM `nfl-data-430702.NFL_Scores.NFL-Plays-2023`
                       """

# Run the query, and return a pandas DataFrame
dry_run_config = bigquery.QueryJobConfig(dry_run=True)
dry_run_query = client.query(nfl_2023_plays_query, job_config=dry_run_config)
print("This query will process {} bytes.".format(dry_run_query.total_bytes_processed))

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**9)
safe_config_query = client.query(nfl_2023_plays_query, job_config=safe_config)

# df_nfl_scores_data = safe_config_query.to_dataframe()

This query will process 11827487 bytes.


In [7]:
df_2023_plays = safe_config_query.to_dataframe()

In [8]:
df_2023_plays.head(10)

,Season,Week,Day,Date,AwayTeam,HomeTeam,Quarter,DriveNumber,TeamWithPossession,IsScoringDrive,PlayNumberInDrive,IsScoringPlay,PlayOutcome,PlayDescription,PlayStart
0,2023,Preseason Week 1,FRI,08/11,Packers,Bengals,1ST QUARTER,1,GB,0,1,0,Kickoff,E.McPherson kicks 67 yards from CIN 35 to GB -...,Kickoff from CIN 35
1,2023,Preseason Week 1,FRI,08/11,Packers,Bengals,1ST QUARTER,1,GB,0,2,0,6 Yard Pass,(14:56) (Shotgun) J.Love pass short right to A...,1st & 10 at GB 18
2,2023,Preseason Week 1,FRI,08/11,Packers,Bengals,1ST QUARTER,1,GB,0,3,0,8 Yard Pass,(14:20) J.Love pass short left to L.Musgrave p...,2nd & 4 at GB 24
3,2023,Preseason Week 1,FRI,08/11,Packers,Bengals,1ST QUARTER,1,GB,0,4,0,3 Yard Run,(13:43) A.Dillon left guard to GB 35 for 3 yar...,1st & 10 at GB 32
4,2023,Preseason Week 1,FRI,08/11,Packers,Bengals,1ST QUARTER,1,GB,0,5,0,Pass Incomplete,(13:06) (Shotgun) J.Love pass incomplete deep ...,2nd & 7 at GB 35
5,2023,Preseason Week 1,FRI,08/11,Packers,Bengals,1ST QUARTER,1,GB,0,6,0,Pass Incomplete,(13:00) (Shotgun) J.Love pass incomplete deep ...,3rd & 7 at GB 35
6,2023,Preseason Week 1,FRI,08/11,Packers,Bengals,1ST QUARTER,1,GB,0,7,0,Punt,"(12:56) P.O'Donnell punts 60 yards to CIN 5, C...",4th & 7 at GB 35
7,2023,Preseason Week 1,FRI,08/11,Packers,Bengals,1ST QUARTER,3,GB,1,1,0,5 Yard Pass,(11:11) (Shotgun) J.Love pass short middle to ...,1st & 10 at CIN 43
8,2023,Preseason Week 1,FRI,08/11,Packers,Bengals,1ST QUARTER,3,GB,1,2,0,Pass Incomplete,(10:37) J.Love pass incomplete short right to ...,2nd & 5 at CIN 38
9,2023,Preseason Week 1,FRI,08/11,Packers,Bengals,1ST QUARTER,3,GB,1,3,0,7 Yard Pass,(10:32) (Shotgun) J.Love pass short right to C...,3rd & 5 at CIN 38


In [9]:
string = df_2023_plays['PlayDescription'].iloc[1]

In [12]:
# Regular expression pattern
pattern = r'\b[A-Z]\.[A-Za-z]+\b'

# Search for the pattern
matches = re.findall(pattern, string)

# Output the matches
print(matches)

['J.Love', 'A.Jones', 'C.Sample']


In [ ]:
# Next step might be to parse out the different 'play outcomes'
# - separate pass / run / kickoff / etc.